In [1]:
# Import modules 

import pandas as pd
from pandas import DataFrame, Series
import json
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
#need to do one more thing to derby: it has too many foreign-language tweets, which apparently 
#have a good chance of referring to soccer games abroad!  I will solve this problem by getting
#rid of all the non-English tweets

from textblob import TextBlob
import pymongo



In [2]:
path = '/home/theresa/Desktop/DataSets/derby924pm.json'
record = [json.loads(line) for line in open(path)]
derby = DataFrame(record)

In [3]:
num_tweets = derby['text'].size

In [4]:
derby.columns

Index(['_id', 'created_at', 'geo', 'source', 'text'], dtype='object')

In [ ]:
derbyEnglish = DataFrame(columns =derby.columns)

In [ ]:
def tweet_to_words( raw_text):
    # Function to convert a raw tweet to a string of words
    # The input is a single string (a raw tweet), and 
    # the output is a single string (a preprocessed tweet)
    
#1. Remove HTML
    review_text = BeautifulSoup(raw_text, "lxml").get_text() 

# 2. Remove non-letters       
    letters_only = re.sub("[^a-zA-Z]+", " ", review_text) 
    
# 3. Convert to lower case and split into individual words
    words = letters_only.lower().split()                             
    
# 4. In Python, searching a set is much faster than searching a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  

# 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops 
                            and 'http' not in w
                            and 'https' not in w
                            and "'" not in w  
                            and not w.startswith('@')
                            and w != 'rt']   
    
# 6. Join the words back into one string separated by space and return the result.
    return( " ".join( meaningful_words ))

In [ ]:
for i in range(min(derby.index), max(derby.index)): #derby.index:#this should be corrected to read num_tweets
    if TextBlob(derby['text'][i]).detect_language() == 'en': ###130119 is the key error for this
    
        derbyEnglish.loc[i] = derby.loc[i]
    derby = derby.drop(i)
    
        
    if i % 1000 == 0 :
            print(i) 


In [ ]:


# Initialize an empty list to hold the clean tweets
clean_tweets_english = []

# Loop over each tweet; create an index i that goes from 0 to the length of the tweet list
for tweet in derbyEnglish['text']:
    # Clean each English tweet, and add the result to the list of clean tweets
    clean_tweets_english.append(tweet_to_words( tweet) )

           


In [ ]:
derbyEnglish['CleanText'] = clean_tweets_english

In [ ]:
derbyEnglish = derbyEnglish.drop(['_id', 'text', 'geo', 'source'], axis=1)  

In [ ]:
derbyEnglish.to_csv('/home/theresa/derby_cleantext.csv',index=False)  